# Detecting Bias in NBA Scorekeeping

In [2]:
import pandas as pd

In [9]:
mpgdf = pd.read_excel('Mean Point guard.xlsx')

In [10]:
mpgdf

,NBA Team,Usage Rate,Points,Rebounds,Assists,Minutes Played,TeamFG%,TeamFG,TeamFGA,Team PPG
0,Boston,34.700000,29.800000,2.700000,5.600000,33.800000,0.445961,39.20,87.900000,105.40
1,Boston Away,33.800000,28.000000,2.400000,6.200000,33.800000,0.430108,36.00,83.700000,105.30
2,Cleveland,29.200000,24.400000,3.300000,6.600000,34.400000,0.445067,39.70,89.200000,104.60
3,Cleveland Away,32.500000,26.100000,3.000000,5.000000,35.700000,0.470056,41.60,88.500000,109.80
4,Toronto,23.900000,21.800000,4.600000,7.000000,37.500000,0.446281,37.80,84.700000,103.20
...,...,...,...,...,...,...,...,...,...,...
57,LAL Away,27.000000,14.600000,3.200000,4.600000,28.300000,0.486175,42.20,86.800000,113.80
58,Phoenix,28.500000,22.100000,4.900000,7.000000,34.100000,0.459054,39.80,86.700000,111.90
59,Phoenix Away,28.400000,20.000000,4.800000,5.700000,31.800000,0.474359,40.70,85.800000,114.70
60,Mean Point Guard,25.776667,18.463333,4.286667,6.653333,31.583333,0.451292,38.59,85.510000,104.02


## Summary Statistics

In [18]:
nbabias.describe()

,Usage Rate,Points,Rebounds,Assists,Minutes Played,TeamFG%,TeamFG,TeamFGA,Team PPG
count,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000
mean,26.028333,18.308333,4.248333,6.415000,31.620000,0.457114,39.025000,85.370877,105.595000
std,5.643723,6.509632,1.886678,1.984324,3.707986,0.014395,1.823215,2.916143,4.430649
min,14.400000,6.200000,1.500000,3.000000,24.300000,0.430108,33.900000,77.500000,94.800000
25%,22.250000,13.350000,3.225000,5.000000,29.100000,0.448420,37.875000,83.550000,102.975000
50%,26.350000,18.050000,3.950000,6.138333,32.500000,0.456287,39.100000,85.555000,105.700000
75%,29.000000,23.000000,4.675000,7.150000,34.325000,0.467137,40.075000,87.475000,109.075000
max,42.900000,32.100000,10.700000,11.400000,37.500000,0.486452,42.500000,90.900000,115.100000


## Importing Logistic Regression Dataset from Stata

In [3]:
nbadta = pd.read_stata('biasinNBAscorekeepingFINAL.dta')

In [4]:
nbadta

,G,Date,Tm,Away,Opp,F,GS,MP,FG,FGA,...,PTS,GmSc,AB,USG,AD,TeamPPG,TeamFG,TripleDouble,TeamPPGPTS,lnPTS
0,1.0,2016-10-26,OKC,1.0,PHI,W (+6),1.0,1900-01-01 12:23:00.000,11.0,21.0,...,32.0,28.4,10.0,31.8,48.5,103.0,0.415,0,71.0,3.465736
1,2.0,2016-10-28,OKC,0.0,PHO,W (+3),1.0,1900-01-01 21:19:00.000,17.0,44.0,...,51.0,32.9,7.0,50.2,58.1,113.0,0.417,1,62.0,3.931826
2,3.0,2016-10-30,OKC,0.0,LAL,W (+17),1.0,1900-01-01 09:39:00.000,11.0,21.0,...,33.0,32.3,9.0,36.0,77.9,113.0,0.474,1,80.0,3.496508
3,4.0,2016-11-02,OKC,1.0,LAC,W (+2),1.0,1900-01-01 08:39:00.000,14.0,30.0,...,35.0,15.9,12.0,56.8,54.8,85.0,0.405,0,50.0,3.555348
4,5.0,2016-11-03,OKC,1.0,GSW,L (-26),1.0,1900-01-01 04:52:00.000,4.0,15.0,...,20.0,13.9,-9.0,39.2,58.7,96.0,0.402,0,76.0,2.995732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78.0,2017-04-05,OKC,1.0,MEM,W (+3),1.0,1900-01-01 13:32:00.000,14.0,25.0,...,45.0,37.0,2.0,44.3,67.0,103.0,0.435,0,58.0,3.806663
78,79.0,2017-04-07,OKC,1.0,PHO,L (-21),1.0,1900-01-01 12:24:00.000,6.0,25.0,...,23.0,10.2,-15.0,42.2,40.4,99.0,0.370,0,76.0,3.135494
79,80.0,2017-04-09,OKC,1.0,DEN,W (+1),1.0,1900-01-01 12:57:00.001,17.0,32.0,...,50.0,45.6,10.0,42.9,72.5,106.0,0.417,1,56.0,3.912023
80,81.0,2017-04-12,OKC,0.0,DEN,L (-6),1.0,1899-12-31 18:19:00.000,2.0,10.0,...,5.0,2.8,-5.0,28.3,55.5,105.0,0.426,0,100.0,1.609438


### Removing an extraneous row

In [5]:
nbadta[0:82]

,G,Date,Tm,Away,Opp,F,GS,MP,FG,FGA,...,PTS,GmSc,AB,USG,AD,TeamPPG,TeamFG,TripleDouble,TeamPPGPTS,lnPTS
0,1.0,2016-10-26,OKC,1.0,PHI,W (+6),1.0,1900-01-01 12:23:00.000,11.0,21.0,...,32.0,28.4,10.0,31.8,48.5,103.0,0.415,0,71.0,3.465736
1,2.0,2016-10-28,OKC,0.0,PHO,W (+3),1.0,1900-01-01 21:19:00.000,17.0,44.0,...,51.0,32.9,7.0,50.2,58.1,113.0,0.417,1,62.0,3.931826
2,3.0,2016-10-30,OKC,0.0,LAL,W (+17),1.0,1900-01-01 09:39:00.000,11.0,21.0,...,33.0,32.3,9.0,36.0,77.9,113.0,0.474,1,80.0,3.496508
3,4.0,2016-11-02,OKC,1.0,LAC,W (+2),1.0,1900-01-01 08:39:00.000,14.0,30.0,...,35.0,15.9,12.0,56.8,54.8,85.0,0.405,0,50.0,3.555348
4,5.0,2016-11-03,OKC,1.0,GSW,L (-26),1.0,1900-01-01 04:52:00.000,4.0,15.0,...,20.0,13.9,-9.0,39.2,58.7,96.0,0.402,0,76.0,2.995732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78.0,2017-04-05,OKC,1.0,MEM,W (+3),1.0,1900-01-01 13:32:00.000,14.0,25.0,...,45.0,37.0,2.0,44.3,67.0,103.0,0.435,0,58.0,3.806663
78,79.0,2017-04-07,OKC,1.0,PHO,L (-21),1.0,1900-01-01 12:24:00.000,6.0,25.0,...,23.0,10.2,-15.0,42.2,40.4,99.0,0.370,0,76.0,3.135494
79,80.0,2017-04-09,OKC,1.0,DEN,W (+1),1.0,1900-01-01 12:57:00.001,17.0,32.0,...,50.0,45.6,10.0,42.9,72.5,106.0,0.417,1,56.0,3.912023
80,81.0,2017-04-12,OKC,0.0,DEN,L (-6),1.0,1899-12-31 18:19:00.000,2.0,10.0,...,5.0,2.8,-5.0,28.3,55.5,105.0,0.426,0,100.0,1.609438


In [6]:
nbadta[0:81]

,G,Date,Tm,Away,Opp,F,GS,MP,FG,FGA,...,PTS,GmSc,AB,USG,AD,TeamPPG,TeamFG,TripleDouble,TeamPPGPTS,lnPTS
0,1.0,2016-10-26,OKC,1.0,PHI,W (+6),1.0,1900-01-01 12:23:00.000,11.0,21.0,...,32.0,28.4,10.0,31.8,48.5,103.0,0.415,0,71.0,3.465736
1,2.0,2016-10-28,OKC,0.0,PHO,W (+3),1.0,1900-01-01 21:19:00.000,17.0,44.0,...,51.0,32.9,7.0,50.2,58.1,113.0,0.417,1,62.0,3.931826
2,3.0,2016-10-30,OKC,0.0,LAL,W (+17),1.0,1900-01-01 09:39:00.000,11.0,21.0,...,33.0,32.3,9.0,36.0,77.9,113.0,0.474,1,80.0,3.496508
3,4.0,2016-11-02,OKC,1.0,LAC,W (+2),1.0,1900-01-01 08:39:00.000,14.0,30.0,...,35.0,15.9,12.0,56.8,54.8,85.0,0.405,0,50.0,3.555348
4,5.0,2016-11-03,OKC,1.0,GSW,L (-26),1.0,1900-01-01 04:52:00.000,4.0,15.0,...,20.0,13.9,-9.0,39.2,58.7,96.0,0.402,0,76.0,2.995732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,77.0,2017-04-04,OKC,0.0,MIL,W (+31),1.0,1900-01-01 03:05:00.000,4.0,12.0,...,12.0,17.6,19.0,26.1,70.0,110.0,0.476,1,98.0,2.484907
77,78.0,2017-04-05,OKC,1.0,MEM,W (+3),1.0,1900-01-01 13:32:00.000,14.0,25.0,...,45.0,37.0,2.0,44.3,67.0,103.0,0.435,0,58.0,3.806663
78,79.0,2017-04-07,OKC,1.0,PHO,L (-21),1.0,1900-01-01 12:24:00.000,6.0,25.0,...,23.0,10.2,-15.0,42.2,40.4,99.0,0.370,0,76.0,3.135494
79,80.0,2017-04-09,OKC,1.0,DEN,W (+1),1.0,1900-01-01 12:57:00.001,17.0,32.0,...,50.0,45.6,10.0,42.9,72.5,106.0,0.417,1,56.0,3.912023
